In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
from collections import Counter
from scipy.signal import find_peaks

In [ ]:
# Set directory with paired read data
samples_dirs = ['merged/merged']

In [ ]:
# Get list of sequencing paths
sequencing_path_list_barcodes = []
for samples_dir in samples_dirs:
    sequencing_path_list_barcodes = sequencing_path_list_barcodes + glob(os.path.join(samples_dir,'*.assembled.fastq'))

In [ ]:
#import reads and show the total and unique read counts
barcodes_now = {}
for (i, filename) in enumerate(sequencing_path_list_barcodes):
    with open(filename, 'r') as f:
        lines = f.readlines()
        barcodes = [line.rstrip() for line in lines[1::4]]
    f.close()
    barcodes_now[os.path.basename(filename).split('.')[0]] = barcodes


# sort keys
oligo_subpools = list(barcodes_now.keys())
oligo_subpools.sort()
    
#read count
read_count = [len(s) for s in [barcodes_now[key] for key in oligo_subpools]] 
fig = plt.figure()
fig.set_size_inches(18.5, 10.5, forward=True)
plt.bar(oligo_subpools, read_count, color='g')
plt.xticks(rotation=90)
plt.title('Total Reads')
plt.show()

#unique reads
barcodes_unique = {}
barcodes_unique_count = {}
barcodes_freq = {}

for key in oligo_subpools:
    barcodes_unique[key] = list(set(barcodes_now[key]))
    barcodes_unique_count[key] = Counter(barcodes_now[key])
    barcodes_freq[key] = {}
    totalreads = len(barcodes_now[key])
    for item, count in barcodes_unique_count[key].items():
        barcodes_freq[key][item] = count/totalreads

#unique read count
unique_read_count = [len(s) for s in [barcodes_unique[key] for key in oligo_subpools]] 
fig = plt.figure()
fig.set_size_inches(18.5, 10.5, forward=True)
plt.bar(oligo_subpools, unique_read_count, color='b')
plt.xticks(rotation=90)
plt.title('Unique Reads')
plt.show()




In [ ]:
# Apply cut-off detection from the previous cell to the first 10 samples in barcodes_unique_count to see if it is working
sample_keys = list(barcodes_unique_count.keys())[:10]
cutoffs = {}
barcodes_real = {}
for key in sample_keys:
    print('\nProcessing', key)
    values = np.array(list(barcodes_unique_count[key].values()), dtype=float)
    # Filter out very small counts
    filtered_values = values[values >= 10**1]
    bins = 10**np.linspace(1, 4, 100)
    hist_counts, bin_edges = np.histogram(filtered_values, bins=bins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    window = 10
    y_smooth = np.convolve(hist_counts, np.ones(window)/window, mode='same')
    peaks, _ = find_peaks(y_smooth)
    # filter peaks by minimum smoothed height
    min_peak_height = 1
    filtered_peaks = peaks[y_smooth[peaks] >= min_peak_height]
    if len(filtered_peaks) < 2:
        print(f"  Not enough modes for {key} after height filter; found {len(filtered_peaks)} peaks. Skipping.")
        continue
    # pick two right-most peaks
    peaks_by_pos = sorted(filtered_peaks, key=lambda p: p, reverse=True)
    rightmost_peak = peaks_by_pos[0]
    second_rightmost_peak = peaks_by_pos[1]
    start = min(rightmost_peak, second_rightmost_peak)
    end = max(rightmost_peak, second_rightmost_peak)
    y_seg = y_smooth[start:end+1]
    min_rel_idx = int(np.argmin(y_seg))
    min_val = y_seg[min_rel_idx]
    seg_range = y_seg.max() - min_val
    rel_frac = 0.05
    if seg_range <= 0:
        tol = min_val
    else:
        tol = min_val + rel_frac * seg_range
    candidate_rel_idxs = np.where(y_seg <= tol)[0]
    if candidate_rel_idxs.size == 0:
        candidate_rel_idxs = np.array([min_rel_idx])
    blocks = []
    if candidate_rel_idxs.size > 0:
        block_start = candidate_rel_idxs[0]
        block_prev = candidate_rel_idxs[0]
        for idx in candidate_rel_idxs[1:]:
            if idx == block_prev + 1:
                block_prev = idx
                continue
            else:
                blocks.append((block_start, block_prev))
                block_start = idx
                block_prev = idx
        blocks.append((block_start, block_prev))
    chosen_block = None
    for bstart, bend in blocks:
        if bstart <= min_rel_idx <= bend:
            chosen_block = (bstart, bend)
            break
    if chosen_block is None:
        lengths = [bend - bstart + 1 for (bstart, bend) in blocks]
        chosen_block = blocks[np.argmax(lengths)]
    bstart, bend = chosen_block
    mid_rel = bstart + (bend - bstart) // 2
    valley_index = start + int(mid_rel)
    cutoff_value = bin_centers[valley_index]
    cutoffs[key] = cutoff_value
    # select barcodes with counts >= cutoff_value
    barcodes_real[key] = [r for r, s in barcodes_unique_count[key].items() if s >= cutoff_value]
    print(f"  Cut-off for {key}: {cutoff_value}")
    # quick plot for this sample
    plt.figure(figsize=(6, 3))
    plt.plot(bin_centers, y_smooth, label='Smoothed', color='C2')
    plt.plot(bin_centers[filtered_peaks], y_smooth[filtered_peaks], 'ro', label='Modes')
    plt.axvspan(bin_centers[start + bstart], bin_centers[start + bend], color='gray', alpha=0.2)
    plt.axvline(cutoff_value, color='red', linestyle='--')
    plt.xscale('log')
    plt.yscale('log')
    plt.title(f"{key} — Cut-off: {cutoff_value:.2f}")
    plt.tight_layout()
    plt.show()

In [ ]:
# Apply cutoff detection to all samples, save filtered sequences per-sample, and write a CSV summary
min_read_threshold = 10  # minimum reads considered when building histograms
min_peak_height = 1     # peaks with smoothed height below this are ignored
bins = 10**np.linspace(1, 4, 100)
window = 10
outdir = 'filtered_sequences'
os.makedirs(outdir, exist_ok=True)
cutoffs_all = {}
barcodes_filtered = {}
rows = []
for key in barcodes_unique_count.keys():
    print('Processing', key)
    values = np.array(list(barcodes_unique_count[key].values()), dtype=float)
    filtered_values = values[values >= min_read_threshold]
    if filtered_values.size == 0:
        print(f'  No counts >= {min_read_threshold} for {key}; using cutoff = {min_read_threshold}')
        cutoff_value = min_read_threshold
    else:
        hist_counts, bin_edges = np.histogram(filtered_values, bins=bins)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        y_smooth = np.convolve(hist_counts, np.ones(window)/window, mode='same')
        peaks, _ = find_peaks(y_smooth)
        # filter peaks by height
        filtered_peaks = peaks[y_smooth[peaks] >= min_peak_height] if peaks.size > 0 else np.array([])
        if filtered_peaks.size >= 2:
            peaks_by_pos = sorted(filtered_peaks, key=lambda p: p, reverse=True)
            rightmost_peak = peaks_by_pos[0]
            second_rightmost_peak = peaks_by_pos[1]
            start = min(rightmost_peak, second_rightmost_peak)
            end = max(rightmost_peak, second_rightmost_peak)
            y_seg = y_smooth[start:end+1]
            min_rel_idx = int(np.argmin(y_seg))
            min_val = y_seg[min_rel_idx]
            seg_range = y_seg.max() - min_val
            rel_frac = 0.05
            if seg_range <= 0:
                tol = min_val
            else:
                tol = min_val + rel_frac * seg_range
            candidate_rel_idxs = np.where(y_seg <= tol)[0]
            if candidate_rel_idxs.size == 0:
                candidate_rel_idxs = np.array([min_rel_idx])
            # group into contiguous blocks
            blocks = []
            block_start = candidate_rel_idxs[0]
            block_prev = candidate_rel_idxs[0]
            for idx in candidate_rel_idxs[1:]:
                if idx == block_prev + 1:
                    block_prev = idx
                else:
                    blocks.append((block_start, block_prev))
                    block_start = idx
                    block_prev = idx
            blocks.append((block_start, block_prev))
            # choose block containing the minimum or the longest block
            chosen_block = None
            for bstart, bend in blocks:
                if bstart <= min_rel_idx <= bend:
                    chosen_block = (bstart, bend)
                    break
            if chosen_block is None:
                lengths = [bend - bstart + 1 for (bstart, bend) in blocks]
                chosen_block = blocks[np.argmax(lengths)]
            bstart, bend = chosen_block
            mid_rel = bstart + (bend - bstart) // 2
            valley_index = start + int(mid_rel)
            cutoff_value = bin_centers[valley_index]
        else:
            # fallback: not enough peaks after filtering -> use minimum threshold
            print(f'  Not enough peaks after height filter for {key} (found {filtered_peaks.size}); using cutoff = {min_read_threshold}')
            cutoff_value = min_read_threshold
    cutoffs_all[key] = cutoff_value
    # collect sequences with counts >= cutoff_value
    filtered_seqs = [r for r, s in barcodes_unique_count[key].items() if s >= cutoff_value]
    # total number of sequences (sum of counts) to the right of the cutoff
    total_seqs = sum(s for r, s in barcodes_unique_count[key].items() if s >= cutoff_value)
    # number of unique sequences passing the cutoff
    n_unique = len(filtered_seqs)
    barcodes_filtered[key] = filtered_seqs
    # write per-sample sequence file (unique sequences)
    outpath = os.path.join(outdir, f"{key}_filtered.txt")
    with open(outpath, 'w') as outfh:
        for seq in filtered_seqs:
            outfh.write(seq + '\n')
    rows.append((key, cutoff_value, total_seqs, n_unique))
# write CSV summary
import csv
csv_path = 'cutoff_filtered_counts.csv'
with open(csv_path, 'w', newline='') as csvf:
    writer = csv.writer(csvf)
    writer.writerow(['sample', 'cutoff', 'n_total_sequences', 'n_unique_sequences'])
    writer.writerows(rows)
print('Wrote per-sample filtered files to', outdir)
print('Wrote CSV summary to', csv_path)

In [ ]:
#Import your oligo sheet or sheets and parse them so that they can be merged with the filtered read counts
# Load the CSV files without headers
master_oligo_df = pd.read_csv('your_first.csv', header=None, names=['name', 'count'])
oligos_df = pd.read_csv('your_second.csv', header=None, names=['name', 'count'])

# Combine the two DataFrames
combined_df = pd.concat([master_oligo_df, oligos_df], ignore_index=True)

# Extract the first and second parts of the row names (before the first `_` and second `_`)
combined_df['group_1'] = combined_df['name'].str.split('_').str[0]
combined_df['group_2'] = combined_df['name'].str.split('_').str[1]

# Replace NaN in group_2 with 'None' for clarity
combined_df['group_2'] = combined_df['group_2'].fillna('None')

# Group by the extracted parts and sum the counts
grouped_counts = combined_df.groupby(['group_1', 'group_2'])['count'].sum().reset_index()

# Rename columns for clarity
grouped_counts.columns = ['group_1', 'group_2', 'total_count']

# Replace sequences in total_count with the count of rows for each group
row_counts = combined_df.groupby(['group_1', 'group_2']).size().reset_index(name='row_count')
grouped_counts = grouped_counts.merge(row_counts, on=['group_1', 'group_2'])
grouped_counts['total_count'] = grouped_counts['row_count']
grouped_counts = grouped_counts.drop(columns=['row_count'])

# Display the resulting DataFrame
print(grouped_counts)

# Optionally, save the grouped counts to a new CSV file
#grouped_counts.to_csv('grouped_counts.csv', index=False)
#print("Grouped counts saved to 'grouped_counts.csv'")

In [ ]:
# Load the cutoff_filtered_counts.csv file
cutoff_filtered_counts_df = pd.read_csv('cutoff_filtered_counts.csv')
# grouped_counts = pd.read_csv('grouped_counts20251120.csv') #if loading from a previously saved file

# Extract the first part of the row names (before the first `_`)
cutoff_filtered_counts_df['group_1'] = cutoff_filtered_counts_df['sample'].str.split('_').str[0]

# Below is an extra step needed if there are exceptions in parsing sample names
# # For samples starting with 'AH', extract the text between the first and second `_` and the second and third `_`, then merge them
# cutoff_filtered_counts_df['group_2'] = cutoff_filtered_counts_df['sample'].apply(
#     lambda x: f"{x.split('_')[1]}_{x.split('_')[2]}" if x.startswith('AH') and len(x.split('_')) > 2 else x.split('_')[1]
# )

# Another extra step for parsing weird sample names
# # Replace any instance of "circRNA" in group_1 with "hairpin"
#cutoff_filtered_counts_df['group_1'] = cutoff_filtered_counts_df['group_1'].replace('circRNA', 'hairpin')

# Update specific rows for group_2
cutoff_filtered_counts_df.loc[250, 'group_2'] = 'current'  # Row 251 (0-indexed)
cutoff_filtered_counts_df.loc[251, 'group_2'] = 'full'    # Row 252 (0-indexed)

# Merge the two DataFrames based on group_1 and group_2
merged_df = pd.merge(cutoff_filtered_counts_df, grouped_counts, on=['group_1', 'group_2'], how='inner')

# Calculate fraction_expected and coverage
merged_df['fraction_expected'] = merged_df['n_unique_sequences'] / merged_df['total_count']
merged_df['coverage'] = merged_df['n_total_sequences'] / merged_df['total_count']

# Display the merged DataFrame
print(merged_df)

#Save the .csv 
merged_df.to_csv('20251120_oligo_lib_results.csv', index=False)